# Autograd

In [4]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import numpy as np
np.random.seed(0)

In [5]:
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn

mean, std = (0.5,), (0.5,)

# Create a transform and normalise data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean, std)
                              ])

# Download FMNIST training dataset and load training data
trainset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download FMNIST test dataset and load test data
testset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [6]:
class FMNIST(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 128)
    self.fc2 = nn.Linear(128,64)
    self.fc3 = nn.Linear(64,10)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    x = F.log_softmax(x, dim=1)
    
    return x
    
#model = FMNIST()   

In [7]:
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

In [11]:
model[1]

ReLU()

In [12]:
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

In [13]:
criterion = nn.NLLLoss()

In [14]:
print('Initial weights:',model[0].weight)

Initial weights: Parameter containing:
tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0295,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0220, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0066,  0.0125,  ...,  0.0286,  0.0350, -0.0105]],
       requires_grad=True)


In [15]:
print('Initial weights gradients:',model[0].weight.grad)

Initial weights gradients: None


In [16]:
output = model(images)
loss = criterion(output, labels)
loss.backward()
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)
        

Initial weights :  Parameter containing:
tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0295,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0220, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0066,  0.0125,  ...,  0.0286,  0.0350, -0.0105]],
       requires_grad=True)
Initial weights gradient :  tensor([[-7.2154e-04, -7.2154e-04, -7.2154e-04,  ..., -7.1856e-04,
         -7.2154e-04, -7.2154e-04],
        [ 1.5117e-03,  1.5168e-03,  1.5292e-03,  ...,  1.5526e-03,
          1.5154e-03,  1.5168e-03],
        [-4.5585e-04, -4.5585e-04, -4.5585e-04,  ..., -4.5585e-04,
         -4.5585e-04, -4.5585e-04],
        ...,
        [-9.6583e-05, -9.6583e-05, -9.6583e-05,  ..., -1.5217e-04,
         -9.6583e-05, -9.6583e-05],
        [-5.6446e-04, -5.6559e-04, -5.5394e-0

In [25]:
model[2].weight.grad

tensor([[ 2.9982e-04,  7.5449e-04, -3.4903e-04,  ..., -4.0108e-03,
         -4.7830e-04,  1.6829e-03],
        [-3.7763e-03, -7.4046e-03, -1.6994e-03,  ...,  4.3599e-03,
         -2.2660e-03, -6.2232e-03],
        [ 0.0000e+00,  3.7159e-04,  2.0443e-05,  ..., -7.8269e-05,
          0.0000e+00, -1.6872e-04],
        ...,
        [ 1.0783e-04,  5.2203e-03,  1.4267e-03,  ...,  5.2378e-04,
         -1.5976e-03, -1.1809e-03],
        [ 2.2829e-03,  5.4146e-03,  4.5451e-04,  ..., -1.0207e-02,
         -2.3595e-03,  2.7769e-03],
        [-3.4852e-03, -5.0445e-03, -1.5482e-03,  ...,  1.0569e-02,
          2.0605e-03, -3.6040e-03]])